# Summary Criterion for Epistasis 

© 2018 Griffin Chure. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT).

--- 

In [5]:
import numpy as np
import pandas as pd
import sympy 
import bokeh.io
sympy.init_printing()
bokeh.io.output_notebook()

Loading BokehJS ...

We are looking for aa useful criterior for understanding and summarizing the degree of epistasis for a given mutant. A useful quantity is the Bohr parameter, which for a given mutant is defined as 

$$
F = k_BT \left(\log p_{act}(c) + \log {R \over N_{NS}} - {\Delta\varepsilon_{RA}\over k_BT}\right) \tag{1},
$$

where $p_{act}(c)$ is the probability of a repressor being in the active state at an inducer concentration $c$. This quantity is defined as 

$$
p_{act}(c) = {\left(1 + {c \over K_A}\right)^n \over \left(1 + {c \over K_A}\right)^n + e^{-\beta\Delta\varepsilon_{AI}}\left(1 + {c \over K_I}\right)^2 }. \tag{2}
$$

We can summarize the change in the Bohr parameter (which can also be interepreted as the free energy) at a given inducer concentration $c$ relative to the wild type repressor as

$$
\Delta F = k_BT \left[ \log {p_{act}^{mut}(c) \over p_{act}^{wt}(c)}  - \left(\Delta\varepsilon_{RA}^{mut} - \Delta\varepsilon_{RA}^{wt}\over k_BT \right)\right]. \tag{3}
$$

Immediately we see the intriguing separation between teh two domains of the repressor with the $p_{act}(c)$ ratio describing changes to the inducer binding domain (and allosteric communication channel) and the DNA binding domain by the $\Delta\Delta\varepsilon_{RA}$. but this is all foragiven concentration of inducer. The $\Delta F$ will change drastically depending on what $c$ we choose. We would like some way in which we can describe the overall effect. 

One option is to integrate Eq. 3 over all $c$, which is worth looking at (although I don't think the integral will converge). Another option is to take the worst case scenario, when $\Delta F$ is maximized. This can be stated mathematically as 

$$
{\partial \Delta F \over \partial c} \bigg\vert_{c=c^*} = 0. \tag{4}
$$

The $p_{act}(c)$ ratio is pretty gnarly, 

$$
{p_{act}^{mut}(c) \over p_{act}^{wt}(c)} = \left[{1 + {c \over K_A^{mut}} \over 1 + {c \over K_A^{wt}}}\right]^n\left[{1 + e^{-\beta\Delta\varepsilon_{AI}^{wt}}\left({1 + {c \over K_I^{wt}}\over 1 + {c \over K_A^{wt}}}\right)^n \over 1 + e^{-\beta\Delta\varepsilon_{AI}^{mut}}\left({1 + {c\over K_I^{mut}} \over 1 + {c \over K_A^{mut}}}\right)^n}\right]. \tag{5}
$$

This is a bit too hardcore to do manually, so we will turn to symbolic computation (i.e. Sympy) to evaluate this.

In [17]:
# Architectural parameters
epRA_mut = sympy.Symbol(r'\Delta\varepsilon_{RA}^{mut}')
epRA_wt = sympy.Symbol(r'\Delta\varepsilon_{RA}^{wt}')

# Allosteric parameters
epAI_mut = sympy.Symbol(r'\Delta\varepsilon_{AI}^{mut}')
epAI_wt = sympy.Symbol(r'\Delta\varepsilon_{AI}^{wt}')
Ka_mut = sympy.Symbol('K_A^{mut}')
Ka_wt = sympy.Symbol('K_A^{wt}')
Ki_mut = sympy.Symbol('K_i^{mut}')
Ki_wt = sympy.Symbol('K_i^{wt}')
n = sympy.Symbol('n')
c = sympy.Symbol('c')

# And finally, the dimensional constant. 
k_BT = sympy.Symbol(r'k_BT')

# Define the states
mut_active_states = (1 + (c / Ka_mut))**n
wt_active_states = (1 + (c / Ka_wt))**n
mut_inactive_states = (1 + (c / Ki_mut))**n
wt_inactive_states = (1 + (c / Ki_wt))**n

# Define the allosteric inactive state term 
wt_numer = (1 + sympy.exp(epAI_wt) * (wt_inactive_states)/(wt_active_states))
mut_denom = (1 + sympy.exp(epAI_mut) * (mut_inactive_states) / (mut_active_states))

# Define the function. 
pact_ratio = sympy.Function(r'\frac{p_{act}^{mut}}{p_{act}^{wt}}')((mut_active_states/wt_active_states) * (wt_numer / mut_denom))

In [20]:
diff = pact_ratio.diff(c)

In [22]:
sympy.simplify(diff)

                  2⋅n                -2⋅n ⎛                  n                
  ⎛K_A__{mut} + c⎞    ⎛K_A__{wt} + c⎞     ⎜  ⎛K_i__{mut} + c⎞                 
n⋅⎜──────────────⎟   ⋅⎜─────────────⎟    ⋅⎜- ⎜──────────────⎟ ⋅(K_A__{mut} - K
  ⎝  K_A__{mut}  ⎠    ⎝  K_A__{wt}  ⎠     ⎝  ⎝  K_i__{mut}  ⎠                 
                                                                              
                                                                              
                                                                              
                                                                              
                                                                              
                                                                              
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                    

In [25]:
sympy.solvers.solve(sympy.Eq(diff, 0), c, positive=True)

NotImplementedError: multiple generators [c, (1 + c/K_A^{mut})**n, (1 + c/K_A^{wt})**n, (1 + c/K_i^{mut})**n, (1 + c/K_i^{wt})**n]
No algorithms are implemented to solve equation (1 + c/K_A^{mut})**n*(1 + c/K_A^{wt})**(-n)*(n*(1 + c/K_A^{wt})**(-n)*(1 + c/K_i^{wt})**n*exp(\Delta\varepsilon_{AI}^{wt})/(K_i^{wt}*(1 + c/K_i^{wt})) - n*(1 + c/K_A^{wt})**(-n)*(1 + c/K_i^{wt})**n*exp(\Delta\varepsilon_{AI}^{wt})/(K_A^{wt}*(1 + c/K_A^{wt})))/(1 + (1 + c/K_A^{mut})**(-n)*(1 + c/K_i^{mut})**n*exp(\Delta\varepsilon_{AI}^{mut})) + (1 + c/K_A^{mut})**n*(1 + c/K_A^{wt})**(-n)*(1 + (1 + c/K_A^{wt})**(-n)*(1 + c/K_i^{wt})**n*exp(\Delta\varepsilon_{AI}^{wt}))*(-n*(1 + c/K_A^{mut})**(-n)*(1 + c/K_i^{mut})**n*exp(\Delta\varepsilon_{AI}^{mut})/(K_i^{mut}*(1 + c/K_i^{mut})) + n*(1 + c/K_A^{mut})**(-n)*(1 + c/K_i^{mut})**n*exp(\Delta\varepsilon_{AI}^{mut})/(K_A^{mut}*(1 + c/K_A^{mut})))/(1 + (1 + c/K_A^{mut})**(-n)*(1 + c/K_i^{mut})**n*exp(\Delta\varepsilon_{AI}^{mut}))**2 - n*(1 + c/K_A^{mut})**n*(1 + c/K